# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [68]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm #stat analysis
import pylab as pl
import numpy as np #arrays


In [69]:
df_raw = pd.read_csv("../assets/admissions.csv") #puts into a pandas object
df = df_raw.dropna() 
print df.head()

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [70]:
# frequency table for prestige and whether or not someone was admitted
from scipy import stats
stats.itemfreq(df['prestige'])

array([[   1.,   61.],
       [   2.,  148.],
       [   3.,  121.],
       [   4.,   67.]])

In [71]:
stats.itemfreq(df['admit'])

array([[  0, 271],
       [  1, 126]])

In [72]:
pd.crosstab(df['admit'], df['prestige'])

prestige,1.0,2.0,3.0,4.0
admit,,,,
0,28,95,93,55
1,33,53,28,12


## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [73]:
dummy_ranks = pd.get_dummies(df.prestige, prefix = 'prestige')
 
dummy_ranks.head()

,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0
0,0,0,1,0
1,0,0,1,0
2,1,0,0,0
3,0,0,0,1
4,0,0,0,1


#### 2.2 When modeling our class variables, how many do we need? 



one less than there are (k-1)

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [74]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_1.0' :])
print handCalc.head()
#iloc takes numerical identifier of the column, loc takes string column name

   admit    gre   gpa  prestige_1.0  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61             0             0             1             0
1      1  660.0  3.67             0             0             1             0
2      1  800.0  4.00             1             0             0             0
3      1  640.0  3.19             0             0             0             1
4      0  520.0  2.93             0             0             0             1


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  


In [75]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted
pd.crosstab(handCalc['admit'], handCalc['prestige_1.0'])

prestige_1.0,0,1
admit,,
0,243,28
1,93,33


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [76]:
33.0 / 28.0

1.1785714285714286

#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [77]:
93.0 / 243.0

0.38271604938271603

#### 3.3 Calculate the odds ratio

In [78]:
(33.0 / 28.0) / (93.0 / 243.0)

3.079493087557604

#### 3.4 Write this finding in a sentence: 

You are more than 3 times more likely to be admitted to grad school if you attended a #1 ranked college than if you did not.

#### 3.5 Print the cross tab for prestige_4

In [79]:
pd.crosstab(handCalc['admit'], handCalc['prestige_4.0'])

prestige_4.0,0,1
admit,,
0,216,55
1,114,12


#### 3.6 Calculate the OR 

In [80]:
(12.0 / 67.0) / (114.0 / 330.0)

0.5184603299293008

#### 3.7 Write this finding in a sentence

It is about half as likely that you get admitted to grad school if you went to a low-prestige school than if you went to a school of higher prestige.

## Part 4. Analysis

In [81]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_ranks.iloc[:, 1:])
print data.head()

   admit    gre   gpa  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61             0             1             0
1      1  660.0  3.67             0             1             0
2      1  800.0  4.00             0             0             0
3      1  640.0  3.19             0             0             1
4      0  520.0  2.93             0             0             1


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [82]:
# manually add the intercept
data['intercept'] = 1.0

#### 4.1 Set the covariates to a variable called train_cols

In [83]:
train_cols = ['gre', 'gpa', 'prestige_2.0', 'prestige_3.0', 'prestige_4.0', 'intercept']

#### 4.2 Fit the model

In [84]:
model = sm.Logit(data['admit'],data[train_cols])

#### 4.3 Print the summary results

In [85]:
result = model.fit()
result.summary()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Wed, 20 Sep 2017   Pseudo R-squ.:                 0.08166
Time:                        04:51:20   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
gre              0.0022      0.001      2.028      0.043    7.44e-05       0.004
gpa              0.7793      0.333      2.344      0.019       0.128       1.431
prestige_2.0    -0.6801      0.317     -2.146      0.032      -1.301      -0.059
prestige_3.0    -1.3387      0.345     -3.882      0.000      -2.015      -0.663
prestige_4.0    -1.5534      0.417     -3.721      0.000      -2.372      -0.735
intercept       -3.8769      1.142     -3.393      0.001      -6.116      -1.638
================================================================================
"""

#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [86]:
np.exp(result.params)

gre             1.002221
gpa             2.180027
prestige_2.0    0.506548
prestige_3.0    0.262192
prestige_4.0    0.211525
intercept       0.020716
dtype: float64

In [87]:
conf = result.conf_int()
conf["OR"] = np.exp(result.params)
conf.columns = ['2.5%', '97.5%', 'OR']
conf.head()

,2.5%,97.5%,OR
gre,0.000074,0.004362,1.002221
gpa,0.127619,1.431056,2.180027
prestige_2.0,-1.301337,-0.058936,0.506548
prestige_3.0,-2.014579,-0.662776,0.262192
prestige_4.0,-2.371624,-0.735197,0.211525


In [88]:
conf["OR"] 

gre             1.002221
gpa             2.180027
prestige_2.0    0.506548
prestige_3.0    0.262192
prestige_4.0    0.211525
intercept       0.020716
Name: OR, dtype: float64

#### 4.5 Interpret the OR of Prestige_2

The odds of getting into grad school from a prestige_2 undergraduate school are half that of getting into grad school from a prestige_1 undergraduate school.

#### 4.6 Interpret the OR of GPA

The odds of being admitted to grad school are twice as good with a higher gpa (?)

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [89]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [90]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


#### 5.1 Recreate the dummy variables

In [91]:
# recreate the dummy variables
combos.columns= ['gre', 'gpa', 'prestige', 'intercept']
df = combos.join(pd.get_dummies(combos['prestige'],prefix='prestige'))
combos.head()


# create a clean data frame for the regression

cols_to_keep = ['gre', 'gpa', 'intercept']
df = df[cols_to_keep].join(df.ix[:, 'prestige_1.0' :])
del df['prestige_1.0']
print df.head()

     gre       gpa  intercept  prestige_2.0  prestige_3.0  prestige_4.0
0  220.0  2.260000        1.0             0             0             0
1  220.0  2.260000        1.0             1             0             0
2  220.0  2.260000        1.0             0             1             0
3  220.0  2.260000        1.0             0             0             1
4  220.0  2.453333        1.0             0             0             0


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  # Remove the CWD from sys.path while we load stuff.


#### 5.2 Make predictions on the enumerated dataset

In [92]:
df['predict'] = result.predict(df[train_cols])
print df.head()



     gre       gpa  intercept  prestige_2.0  prestige_3.0  prestige_4.0  \
0  220.0  2.260000        1.0             0             0             0   
1  220.0  2.260000        1.0             1             0             0   
2  220.0  2.260000        1.0             0             1             0   
3  220.0  2.260000        1.0             0             0             1   
4  220.0  2.453333        1.0             0             0             0   

    predict  
0  0.164173  
1  0.090492  
2  0.048977  
3  0.039890  
4  0.185907  


#### 5.3 Interpret findings for the last 4 observations

In [93]:
df[-4:]

,gre,gpa,intercept,prestige_2.0,prestige_3.0,prestige_4.0,predict
396,800.0,4.0,1.0,0,0,0,0.734040
397,800.0,4.0,1.0,1,0,0,0.582995
398,800.0,4.0,1.0,0,1,0,0.419833
399,800.0,4.0,1.0,0,0,1,0.368608


Based on this model, there is a 73% chance that a student who had the highest possible GRE, GPA, and prestige level for undergrad gets admitted to grad school. For perfect GRE score and GPA with undergrad prestige of 2, the model predicts a 58% chance of getting into grad school. For a student with perfect GRE and GPA but who attended a 3rd and 4th level prestige undergraduate school, there is a 42% and 37% chance of admission into grad school respectively.

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.